In [2]:
import pandas as pd 
import numpy as np 
import timeit 

df = pd.read_csv("data/census.csv")
df.head(5)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


## Method Chaining - pandorable
    - every method on object returns a reference to that object.
    - Beauty > different operation into one line/statement 

In [5]:
new_df = df[df['SUMLEV']==50]
new_df.set_index(['STNAME', 'CTYNAME']).rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})

SUMLEV  REGION  ...  RNETMIG2014  RNETMIG2015
STNAME  CTYNAME                            ...                          
Alabama Autauga County         50       3  ...     2.592270    -2.187333
        Baldwin County         50       3  ...    20.317142    18.293499
        Barbour County         50       3  ...    -3.978583   -10.543299
        Bibb County            50       3  ...     0.754533     1.107861
        Blount County          50       3  ...    -1.577232    -0.884411
...                           ...     ...  ...          ...          ...
Wyoming Sweetwater County      50       4  ...   -14.075283   -14.070195
        Teton County           50       4  ...    16.308671     1.520747
        Uinta County           50       4  ...   -14.740608   -12.606351
        Washakie County        50       4  ...   -18.020168     1.441961
        Weston County          50       4  ...     1.533635     6.935294

[3142 rows x 98 columns]

In [11]:
# Comparison 
def their_approach():
    global df 
    return df.where(df['SUMLEV']==50).dropna().set_index('STNAME','CTYNAME').rename(columns={"POPESTIMATE2010":"Population Estimate 2010"})
df = pd.read_csv("datasets/census.csv")
t1 = timeit.timeit(their_approach,number=10)

In [15]:
def my_approach():
    global df
    new_df = df[df['SUMLEV']==50]
    new_df.set_index(['STNAME', 'CTYNAME'], inplace=True)
    return new_df.rename(columns={'ESTIMATESBASE2010': 'Estimates Base 2010'})
df = pd.read_csv("datasets/census.csv")
t2 = timeit.timeit(my_approach,number=10) 
print(t2)

0.18882739999980913


In [14]:
diff = t1 - t2
print(diff)

0.1642113000000336


In [3]:
def min_max(row):
    data = row[[
        'POPESTIMATE2010',
        'POPESTIMATE2011',
        'POPESTIMATE2012',
        'POPESTIMATE2013',
        'POPESTIMATE2014',
        'POPESTIMATE2015']]
    return pd.Series({'min':np.min(data), 'max': np.max(data)})

In [6]:
df.apply(min_max, axis='columns').head()

,min,max
0,4785161,4858979
1,54660,55347
2,183193,203709
3,26489,27341
4,22512,22861


In [8]:
rows = ['POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013','POPESTIMATE2014', 
        'POPESTIMATE2015']
df.apply(lambda x: np.max(x[rows]),axis=1).head()

0    4858979
1      55347
2     203709
3      27341
4      22861
dtype: int64

In [9]:
def get_state_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    
    if x in northeast:
        return "Northeast"
    elif x in midwest:
        return "Midwest"
    elif x in south:
        return "South"
    else:
        return "West"

In [12]:
df['state_region'] = df['STNAME'].apply(lambda x: get_state_region(x))

In [13]:
df[['state_region','STNAME']].head()

,state_region,STNAME
0,South,Alabama
1,South,Alabama
2,South,Alabama
3,South,Alabama
4,South,Alabama


In [14]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>